In [14]:
import pandas as pd
import os
import shutil
import re

def split_csv_by_customer_id(
    input_csv_path: str,
    customer_id_column: str,
    output_dir: str = 'customer_files',
    encoding: str = 'utf-8' # Common encoding for Traditional Chinese
    ):
    """
    Reads a CSV file, splits it into multiple CSV files based on unique values
    in the specified customer ID column, saves them into a directory,
    and creates a zip archive of that directory.

    Args:
        input_csv_path (str): Path to the input CSV file.
        customer_id_column (str): The exact name of the column containing customer IDs.
        output_dir (str): The name of the directory to save the individual CSV files.
        encoding (str): The character encoding to use when reading/writing CSVs.
    """
    print(f"Starting the process for file: {input_csv_path}")
    print(f"Using Customer ID column: '{customer_id_column}'")
    print(f"Attempting to use encoding: '{encoding}'")
    print(f"Output directory: '{output_dir}'")

    try:
        # --- 1. Read the input CSV file ---
        print(f"Reading CSV file: {input_csv_path}...")
        # Explicitly set dtype for the ID column to string if possible,
        # otherwise pandas might infer numeric types incorrectly.
        # If the column doesn't exist, this will likely fail later anyway.
        try:
            df = pd.read_csv(input_csv_path, encoding=encoding, dtype={customer_id_column: str})
        except ValueError as ve:
            # If specifying dtype fails (e.g., column not found yet), read without it
            print(f"Warning: Could not pre-specify dtype for '{customer_id_column}'. Reading without dtype specification. Error: {ve}")
            df = pd.read_csv(input_csv_path, encoding=encoding)

        print("Successfully read CSV file.")

        # --- 2. Validate Customer ID column ---
        if customer_id_column not in df.columns:
            print(f"Error: Column '{customer_id_column}' not found in the CSV file.")
            print(f"Available columns are: {df.columns.tolist()}")
            return
        print(f"Validated column '{customer_id_column}'.")

        # --- 3. Create output directory ---
        if os.path.exists(output_dir):
            print(f"Output directory '{output_dir}' already exists. Removing existing directory...")
            shutil.rmtree(output_dir) # Remove old directory to avoid mixing files
        os.makedirs(output_dir)
        print(f"Created output directory: '{output_dir}'")

        # --- 4. Get unique customer IDs ---
        # Convert column to string explicitly and handle potential NaN values
        unique_ids = df[customer_id_column].astype(str).unique()
        # Remove potential 'nan' strings if they resulted from conversion
        unique_ids = [id_val for id_val in unique_ids if id_val.lower() != 'nan']

        print(f"Found {len(unique_ids)} unique customer IDs.")
        if not unique_ids:
            print("Warning: No unique customer IDs found. No files will be created.")
            return

        # --- 5. Split data and save to individual files ---
        print("Splitting data and saving files...")
        file_count = 0
        for customer_id in unique_ids:
            # Filter dataframe for the current customer ID
            # Ensure comparison is done correctly (string vs string)
            customer_df = df[df[customer_id_column].astype(str) == customer_id]

            if not customer_df.empty:
                # Sanitize customer_id for use as a filename
                # Remove/replace characters invalid for filenames
                sanitized_id = re.sub(r'[\\/*?:"<>|]', '_', str(customer_id))
                # Ensure filename isn't empty after sanitization
                if not sanitized_id:
                    sanitized_id = f"customer_{file_count+1}" # Fallback filename

                output_filename = f"{sanitized_id}.csv"
                output_filepath = os.path.join(output_dir, output_filename)

                # Save the filtered data to a new CSV
                customer_df.to_csv(output_filepath, index=False, encoding=encoding)
                file_count += 1

        print(f"Successfully created {file_count} individual customer CSV files in '{output_dir}'.")

        # --- 6. Zip the output directory ---
        if file_count > 0:
            zip_filename = f"{output_dir}.zip"
            print(f"Creating zip archive: {zip_filename}...")
            shutil.make_archive(output_dir, 'zip', output_dir)
            print(f"Successfully created zip archive: {zip_filename}")
        else:
            print("No files were created, skipping zip archive creation.")

    except FileNotFoundError:
        print(f"Error: The input file '{input_csv_path}' was not found.")
    except UnicodeDecodeError:
        print(f"Error: Could not decode the file using encoding '{encoding}'.")
        print("Please verify the file encoding and provide the correct one.")
    except KeyError:
        # This might catch cases where the column name was slightly off even if it passed initial check
         print(f"Error: A KeyError occurred, likely related to accessing column '{customer_id_column}'.")
         print("Please double-check the exact column name.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# --- How to use the function ---
# 1. Make sure the input CSV file is available in the environment.
# 2. Adjust the parameters if needed (especially customer_id_column).
input_file = '/home/hansen/workspace/hackathon/hackson-voice-agent/data/data.csv'
assumed_id_column = '客代' # IMPORTANT: Change this if '客代' is incorrect!

# Run the function
split_csv_by_customer_id(input_csv_path=input_file, customer_id_column=assumed_id_column)



Starting the process for file: /home/hansen/workspace/hackathon/hackson-voice-agent/data/data.csv
Using Customer ID column: '客代'
Attempting to use encoding: 'utf-8'
Output directory: 'customer_files'
Reading CSV file: /home/hansen/workspace/hackathon/hackson-voice-agent/data/data.csv...
Successfully read CSV file.
Validated column '客代'.
Created output directory: 'customer_files'
Found 100 unique customer IDs.
Splitting data and saving files...
Successfully created 100 individual customer CSV files in 'customer_files'.
Creating zip archive: customer_files.zip...
Successfully created zip archive: customer_files.zip


In [15]:
import pandas as pd
import io
import glob
import shutil

def format_customer_data_to_txt(csv_data_string: str) -> str:
    """
    Transforms CSV data for a single customer into a structured TXT format.

    Args:
        csv_data_string: A string containing the CSV data for one customer.
                         Expected headers: 客代,標籤分類1,標籤分類2,標籤名稱,標籤值

    Returns:
        A string containing the formatted customer profile in TXT format.
        Returns an error message string if processing fails.
    """
    try:
        # Use StringIO to read the CSV data string into a pandas DataFrame
        data_io = io.StringIO(csv_data_string)
        df = pd.read_csv(data_io)

        # --- Basic Validation ---
        required_columns = ['客代', '標籤分類1', '標籤名稱', '標籤值']
        if not all(col in df.columns for col in required_columns):
            return f"Error: Input CSV data is missing one or more required columns: {required_columns}"

        if df.empty:
            return "Error: Input CSV data is empty."

        # Get the customer ID (should be the same for all rows)
        customer_id = df['客代'].iloc[0]

        # --- Group data and aggregate values ---
        # Group by category and attribute name, collect values into a list
        grouped = df.groupby(['標籤分類1', '標籤名稱'])['標籤值'].apply(list).reset_index()

        # --- Build the output TXT string ---
        output_lines = []
        output_lines.append(f"Customer Profile: 客代 {customer_id}")
        output_lines.append("") # Add a blank line

        # Iterate through the primary categories (標籤分類1)
        for category1 in grouped['標籤分類1'].unique():
            output_lines.append(f"{category1}:")
            # Get attributes and values within this category
            category_data = grouped[grouped['標籤分類1'] == category1]
            for _, row in category_data.iterrows():
                key = row['標籤名稱']
                # Join list of values with a comma, convert items to string first
                value = ", ".join(map(str, row['標籤值']))
                output_lines.append(f"  {key}: {value}") # Indent attributes
            output_lines.append("") # Add a blank line after each category section

        return "\n".join(output_lines).strip() # Join lines and remove trailing whitespace

    except pd.errors.EmptyDataError:
        return "Error: No columns to parse from CSV data string."
    except Exception as e:
        return f"An unexpected error occurred during formatting: {e}"

def process_csv_folder_to_txt(
    input_csv_dir: str,
    output_txt_dir: str,
    csv_encoding: str = 'big5' # Encoding to read the source CSVs
    ):
    """
    Processes a folder of customer CSV files, formats each using
    format_customer_data_to_txt, and saves the results as TXT files
    in a specified output directory.

    Args:
        input_csv_dir (str): Path to the directory containing individual customer CSV files.
        output_txt_dir (str): Path to the directory where formatted TXT files will be saved.
        csv_encoding (str): Encoding used to read the input CSV files.
    """
    print(f"Starting batch processing...")
    print(f"Input CSV directory: '{input_csv_dir}'")
    print(f"Output TXT directory: '{output_txt_dir}'")
    print(f"Using CSV reading encoding: '{csv_encoding}'")

    # --- Create output directory ---
    if not os.path.exists(output_txt_dir):
        os.makedirs(output_txt_dir)
        print(f"Created output directory: '{output_txt_dir}'")
    else:
        print(f"Output directory '{output_txt_dir}' already exists.")

    # --- Find all CSV files in the input directory ---
    csv_files = glob.glob(os.path.join(input_csv_dir, '*.csv'))

    if not csv_files:
        print(f"Warning: No CSV files found in '{input_csv_dir}'.")
        return

    print(f"Found {len(csv_files)} CSV files to process.")
    success_count = 0
    error_count = 0

    # --- Loop through each CSV file ---
    for csv_filepath in csv_files:
        base_filename = os.path.basename(csv_filepath)
        print(f"Processing '{base_filename}'...")

        try:
            # --- Read the CSV file content into a string ---
            with open(csv_filepath, 'r', encoding=csv_encoding) as f:
                csv_content_string = f.read()

            # --- Format the data using the helper function ---
            formatted_text = format_customer_data_to_txt(csv_content_string)

            # --- Check for errors from the formatting function ---
            if formatted_text.startswith("Error:"):
                print(f"  Error formatting '{base_filename}': {formatted_text}")
                error_count += 1
                continue # Skip to the next file

            # --- Construct output TXT filename and path ---
            txt_filename = os.path.splitext(base_filename)[0] + '.txt'
            output_filepath = os.path.join(output_txt_dir, txt_filename)

            # --- Write the formatted text to the TXT file ---
            # Use UTF-8 for writing TXT files, as it's widely compatible
            with open(output_filepath, 'w', encoding='utf-8') as f_out:
                f_out.write(formatted_text)

            success_count += 1
            # print(f"  Successfully created '{txt_filename}'") # Optional: more verbose logging

        except FileNotFoundError:
            print(f"  Error: File not found '{csv_filepath}' (should not happen with glob).")
            error_count += 1
        except UnicodeDecodeError:
            print(f"  Error: Could not decode '{base_filename}' using encoding '{csv_encoding}'. Try a different encoding.")
            error_count += 1
        except Exception as e:
            print(f"  An unexpected error occurred processing '{base_filename}': {e}")
            error_count += 1

    print("\nBatch processing finished.")
    print(f"Successfully processed: {success_count} files.")
    print(f"Errors encountered: {error_count} files.")

    # --- Optional: Zip the output directory ---
    if success_count > 0:
        try:
            zip_filename = f"{output_txt_dir}.zip"
            print(f"\nCreating zip archive: {zip_filename}...")
            shutil.make_archive(output_txt_dir, 'zip', output_txt_dir)
            print(f"Successfully created zip archive: {zip_filename}")
        except Exception as e:
            print(f"Could not create zip file: {e}")

# --- Example Usage with your provided data ---
sample_customer_csv_data = """客代,標籤分類1,標籤分類2,標籤名稱,標籤值
23,基本類別,基本類別,性別,女
23,基本類別,基本類別,年齡區間,60-69
23,基本類別,基本類別,星座,射手座
23,基本類別,基本類別,居住縣市,台中市太平區
23,基本類別,基本類別,會員等級,A級會員
23,基本類別,基本類別,會員年資分組,20年以上-25年以下
23,基本類別,基本類別,職業,退休族
23,保健,保健功效,可推薦商品類別,美容養顏
23,保健,保健功效,可推薦商品類別,強化靈活關節
23,保健,保健功效,可推薦商品類別,提升調節免疫力
23,保健,保健功效,可推薦商品類別,體重管理
23,保健,高健康意識,是否有高健康意識,是
23,宗教商品,宗教風水偏好,有無宗教商品偏好,有
23,生活,生活用品偏好,有無生活用品偏好,有
23,美容,美容偏好,有無美容偏好,有
23,美容,美容偏好,美妝保養類型偏好,護膚SPA
23,食品,食品偏好,有無食品偏好,有
23,寵物,寵物偏好,有無養寵物,無"""

formatted_text = format_customer_data_to_txt(sample_customer_csv_data)
print(formatted_text)


Customer Profile: 客代 23

保健:
  可推薦商品類別: 美容養顏, 強化靈活關節, 提升調節免疫力, 體重管理
  是否有高健康意識: 是

基本類別:
  居住縣市: 台中市太平區
  年齡區間: 60-69
  性別: 女
  星座: 射手座
  會員年資分組: 20年以上-25年以下
  會員等級: A級會員
  職業: 退休族

宗教商品:
  有無宗教商品偏好: 有

寵物:
  有無養寵物: 無

生活:
  有無生活用品偏好: 有

美容:
  有無美容偏好: 有
  美妝保養類型偏好: 護膚SPA

食品:
  有無食品偏好: 有


In [16]:
input_directory = '/home/hansen/workspace/hackathon/hackson-voice-agent/data/customer_files_csv'

# 2. Set the desired output directory name for the TXT files.
output_directory = '/home/hansen/workspace/hackathon/hackson-voice-agent/data/customer_files_txt'

# 3. Specify the encoding of your input CSV files (adjust if 'big5' is wrong).
input_encoding = 'utf-8'

# 4. Run the processing function.
process_csv_folder_to_txt(
    input_csv_dir=input_directory,
    output_txt_dir=output_directory,
    csv_encoding=input_encoding
)


Starting batch processing...
Input CSV directory: '/home/hansen/workspace/hackathon/hackson-voice-agent/data/customer_files_csv'
Output TXT directory: '/home/hansen/workspace/hackathon/hackson-voice-agent/data/customer_files_txt'
Using CSV reading encoding: 'utf-8'
Output directory '/home/hansen/workspace/hackathon/hackson-voice-agent/data/customer_files_txt' already exists.
Found 100 CSV files to process.
Processing '41.csv'...
Processing '15.csv'...
Processing '76.csv'...
Processing '16.csv'...
Processing '56.csv'...
Processing '27.csv'...
Processing '85.csv'...
Processing '92.csv'...
Processing '79.csv'...
Processing '11.csv'...
Processing '12.csv'...
Processing '90.csv'...
Processing '66.csv'...
Processing '8.csv'...
Processing '72.csv'...
Processing '89.csv'...
Processing '23.csv'...
Processing '30.csv'...
Processing '46.csv'...
Processing '20.csv'...
Processing '55.csv'...
Processing '22.csv'...
Processing '65.csv'...
Processing '82.csv'...
Processing '51.csv'...
Processing '67.c